Eli Chesnut and Kiersten Wener

In [4]:
##
import kagglehub
import pandas as pd
import os
import numpy as np

# Download latest version
path = kagglehub.dataset_download("datatattle/covid-19-nlp-text-classification")
csv_train_file_path = os.path.join(path, "Corona_NLP_train.csv")
csv_test_file_path = os.path.join(path, "Corona_NLP_test.csv")
print("Path to dataset files:", path)


# file = f"{path}/Corona_NLP_train.csv"
# df = pd.read_csv(file)
# df.head()

100%|██████████| 4.38M/4.38M [00:02<00:00, 1.56MB/s]

Extracting files...
Path to dataset files: /Users/kierstenwener/.cache/kagglehub/datasets/datatattle/covid-19-nlp-text-classification/versions/1


In [5]:
df_train = pd.read_csv(csv_train_file_path, encoding='ISO-8859-1') #can infer col names
df_test = pd.read_csv(csv_test_file_path, encoding='ISO-8859-1')

df_train.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [6]:
columns_to_drop = ["UserName", "ScreenName", "Location", "TweetAt"]
df_train = df_train.drop(columns=columns_to_drop)
df_test = df_test.drop(columns=columns_to_drop)
df = pd.concat([df_test, df_train], ignore_index=True)
df.head()

,OriginalTweet,Sentiment
0,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,When I couldn't find hand sanitizer at Fred Me...,Positive
2,Find out how you can protect yourself and love...,Extremely Positive
3,#Panic buying hits #NewYork City as anxious sh...,Negative
4,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral


In [7]:
df["tweet_length"] = df["OriginalTweet"].apply(len)  # Add a column for tweet lengths
max_length = df["tweet_length"].max()         # Get the maximum length
longest_tweet = df.loc[df["tweet_length"] == max_length, "OriginalTweet"].iloc[0]  # Get the longest tweet

print(max_length)
print(longest_tweet)

355
Crude oil dropped to its lowest in 17 years in Asia as #coronavirus in #US and #EU shows little sign of slowing.

Noon prices:
#SPX500Â2504.3
#NAS100Â7507.1
#WTIÂ20.24
#GOLDÂ1618.08
#SILVERÂ14.118

Watch these prices closely: https://t.co/WuiEEfSNOj

#COVID19 #CoronaVirusOutbreak https://t.co/j68MZIDXAd https://t.co/9MyqGxutUU


Based on this quick test to determine what the longest tweet is we can now set our max review length

In [8]:
max_review_length = 400
df = df.drop(columns="tweet_length")
df.head()


,OriginalTweet,Sentiment
0,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,When I couldn't find hand sanitizer at Fred Me...,Positive
2,Find out how you can protect yourself and love...,Extremely Positive
3,#Panic buying hits #NewYork City as anxious sh...,Negative
4,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral


In [9]:
num_duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {num_duplicates}")
#no duplicates


Number of duplicate rows: 0


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44955 entries, 0 to 44954
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   OriginalTweet  44955 non-null  object
 1   Sentiment      44955 non-null  object
dtypes: object(2)
memory usage: 702.6+ KB


In [11]:
u = df["Sentiment"].unique()
print(u)

['Extremely Negative' 'Positive' 'Extremely Positive' 'Negative' 'Neutral']


In [12]:
df["Sentiment"] = df["Sentiment"].apply(lambda x: "Negative" if x == "Extremely Negative" else "Positive" if x == "Extremely Positive" else x)

u = df["Sentiment"].unique()
print(u)

['Negative' 'Positive' 'Neutral']


In [13]:
df_ohe = pd.get_dummies(df, columns=["Sentiment"])
df_ohe.head()

,OriginalTweet,Sentiment_Negative,Sentiment_Neutral,Sentiment_Positive
0,TRENDING: New Yorkers encounter empty supermar...,True,False,False
1,When I couldn't find hand sanitizer at Fred Me...,False,False,True
2,Find out how you can protect yourself and love...,False,False,True
3,#Panic buying hits #NewYork City as anxious sh...,True,False,False
4,#toiletpaper #dunnypaper #coronavirus #coronav...,False,True,False


In [15]:
from sklearn.model_selection import train_test_split
targets = ["Sentiment_Negative","Sentiment_Neutral", "Sentiment_Positive"]
X = df_ohe.drop(columns=targets)
y = df_ohe[targets]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [18]:
import setuptools.dist
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer


# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)  # Fit the tokenizer on the training data

# Convert the text to sequences of integers
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

# Pad the sequences to ensure uniform length
max_review_length = 400
X_train_padded = sequence.pad_sequences(X_train_sequences, maxlen=max_review_length)
X_test_padded = sequence.pad_sequences(X_test_sequences, maxlen=max_review_length)


In [19]:
from tensorflow.keras.layers import Embedding, Input

EMBED_SIZE = 50
# the input is a list of integers, 500 long
sequence_input = Input(shape=(X_train.shape[1], ))

embedded_sequences = Embedding(400, # input dimension (max int of OHE)
                EMBED_SIZE, # output dimension size
                input_length=max_review_length)(sequence_input) # number of words in each sequence

/Users/kierstenwener/Desktop/MLLab7/.venv/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


We have now created the training and testing sets for our multi class classification. The dataset that we are using has several columns that are not relevant for this lab and we disgard them. The data is already split up into a training and testing CSV, but for the purposes of doing our own splitting these were combined into one dataframe. We then edited the targets to simplify the classification before one hot encoding them. Before padding the sentences we used the keras tokenizer giving us a word index. We then used Embedding. 

The F1 score is a particularly useful metric for classifying Covid-19 tweet sentiment into negative, positive, and neutral categories due to its ability to balance both precision and recall. In sentiment analysis tasks, especially with imbalanced datasets, the F1 score ensures that the model doesn't simply favor the majority class at the expense of identifying less frequent but equally important categories. Since misclassifying a sentiment can have significant implications, the F1 score helps optimize the trade-off between precision (avoiding false positives) and recall (avoiding false negatives), providing a more comprehensive evaluation of the model's performance across all sentiment classes. By focusing on both false positives and false negatives, the F1 score ensures that the model is robust and effective at correctly predicting sentiments for all classes, making it ideal for real-world applications such as analyzing public opinion during a crisis like the Covid-19 pandemic.


Train-test split is a solid way to divide up a dataset because it gives a simple yet effective way to evaluate how well our model generalizes to new, unseen data. By splitting the data into two sets ensures that our model doesn't just memorize the data and reduces overfitting. It’s especially helpful for tasks like classifying Covid-19 tweet sentiments, where the tweets might vary widely in content and sentiment. Using train_test_split with shuffling helps mix things up, so we are not training on a block of tweets with one dominant sentiment and testing on another block with a different sentiment. This way we get a better picture of how our model will perform across all types of sentiments. This method also gives us an unbiased estimate of how well the model will do in real-world situations, where the data distribution is unpredictable.

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.layers import Embedding, Input, Concatenate
from tensorflow.keras.layers import Subtract
from tensorflow.keras.utils import plot_model
import tensorflow as tf

EMBED_SIZE = 50
# the input is a list of integers, 500 long
sequence_input = Input(shape=(X_train.shape[1], ))

# this will reduce the input dimension from VOCAB_SIZE to 50 for each word
# the lenght will be the maximum number of words in a document, so 500
embedded_sequences = Embedding(top_words, # input dimension (max int of OHE)
                EMBED_SIZE, # output dimension size
                input_length=max_review_length)(sequence_input) # number of words in each sequence
 
# starting sequence size is 500 (words) by 50 (embedded features)
x = Conv1D(64, 5, activation='relu',
           kernel_initializer='he_uniform')(embedded_sequences)

# after conv, length becomes: 500-4=496 
# so overall size is 496 by 64

# now pool across time
x = MaxPooling1D(5)(x)# after max pool, 496/5 -> 99 by 64
x = Dropout(0.2)(x)

# extract additional features
x = Conv1D(64, 5, activation='relu',
           kernel_initializer='he_uniform')(x)

# new size is 95 after the conovlutions
x = MaxPooling1D(5)(x) # after max pool, size is 95/5 = 19 by 64
x = Dropout(0.2)(x)

# more features through CNN processing!
x = Conv1D(64, 5, activation='relu',
           kernel_initializer='he_uniform')(x)
x = Dropout(0.2)(x)

# after convolution, size becomes 15 elements long
# Take the mean of these elements across features, result is 64 elements
x_mean = GlobalAveragePooling1D()(x) # this is the size to globally flatten 

# Take the variance of these elements across features, result is 64 elements
x_tmp = Subtract()([x,x_mean])
x_std = GlobalAveragePooling1D()(x_tmp**2)

x = Concatenate(name='concat_1')([x_mean,x_std])

x = Dense(64, activation='relu',
          kernel_initializer='he_uniform')(x)

x = Dropout(0.2)(x)

preds = Dense(NUM_CLASSES, activation='sigmoid',
              kernel_initializer='glorot_uniform')(x)

model = Model(sequence_input, preds)

print(model.summary())

# you will need to install pydot properly on your machine to get this running
plot_model(
    model, to_file='model.png', show_shapes=True, show_layer_names=True,
    rankdir='LR', expand_nested=False, dpi=96
)